In [97]:
import os, sys, shutil

from dotenv import load_dotenv

load_dotenv("../../_apikeys.env")
api_key = os.getenv("DoogieOpenaiKey")
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
#
# Step 1-1: get test documents
#
import urllib.request

urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="06_07_test.pdf")

In [98]:
#
# Step 1-2: load and split documents
# 17~20 sec
#
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader( "06_07_test.pdf" )
pages = loader.load_and_split()  # about 17s
#pages = loader.load()  # about 16s, include empty page


In [99]:
print( 'type loader,pages:', type(loader), type(pages) )
print( 'type pages[0]s:', type(pages[0]), type(pages[0].page_content))
print( 'size/len loader,pages:', sys.getsizeof(loader), len(pages) )
print( 'size/len pages[0]s:', sys.getsizeof(pages[0]), len(pages[0].page_content) )
print( 'pages Max:', max(len(apage.page_content) for apage in pages ) )
print( 'pages Min:', min(len(apage.page_content) for apage in pages ) )
print( 'pages Sum:', sum(len(apage.page_content) for apage in pages ) )
print( '#of chunks:', len(pages) )
print( 'Chunk Avg:', sum(len(apage.page_content) for apage in pages ) / len(pages) )

type loader,pages: <class 'langchain_community.document_loaders.pdf.PyPDFLoader'> <class 'list'>
type pages[0]s: <class 'langchain_core.documents.base.Document'> <class 'str'>
size/len loader,pages: 48 445
size/len pages[0]s: 72 57
pages Max: 1640
pages Min: 6
pages Sum: 372229
#of chunks: 445
Chunk Avg: 836.4696629213483


In [ ]:
#print( pages[0] )
#print( pages[0].page_content)
#print( "example:", pages[6].page_content[:50] )
#print( pages[442].page_content )

In [100]:
# Step 2: case1:
# RecursiveCharacterTextSpilter를 사용함
#
from langchain.text_splitter import RecursiveCharacterTextSplitter

textSplitter = RecursiveCharacterTextSplitter(
    chunk_size= 1000,
    chunk_overlap=0,
    separators=["\n\n", "\n", " ", ""]
)

splitedDocs = textSplitter.split_documents( pages )

In [101]:
print( 'type split,splitDocs:', type(textSplitter), type(splitedDocs) )
print( 'type splitedDocs[0]s:', type(splitedDocs[0]), type(splitedDocs[0].page_content) )
print( 'size/len split,splitDocs:', sys.getsizeof(textSplitter), len(splitedDocs) )
print( 'size/len splitedDocs[0]s:', sys.getsizeof(splitedDocs[0]), len(splitedDocs[0].page_content) )
print( 'splitedDocs Max:', max(len(apage.page_content) for apage in splitedDocs ) )
print( 'splitedDocs Min:', min(len(apage.page_content) for apage in splitedDocs ) )
print( 'splitedDocs Sum:', sum(len(apage.page_content) for apage in splitedDocs ) )
print( '#of chunks:', len(splitedDocs) )
print( 'Chunk Avg:', sum(len(apage.page_content) for apage in splitedDocs ) / len(splitedDocs) )

type split,splitDocs: <class 'langchain_text_splitters.character.RecursiveCharacterTextSplitter'> <class 'list'>
type splitedDocs[0]s: <class 'langchain_core.documents.base.Document'> <class 'str'>
size/len split,splitDocs: 48 496
size/len splitedDocs[0]s: 72 57
splitedDocs Max: 1000
splitedDocs Min: 6
splitedDocs Sum: 372148
#of chunks: 496
Chunk Avg: 750.2983870967741


In [104]:
#
# DONT USE THIS CODE !!! use Step3-2 Code !!!
# Step3-1: Loading splitedDocs to chroma
# Error case: "Requested 367501 tokens, max 300000 tokens per request"
#
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
#from langchain.vectorstores import Chroma

myEmbeddings = OpenAIEmbeddings()
print("current OpenAIEmbeddings().model=", myEmbeddings.model)
myEmbeddings = OpenAIEmbeddings(model="text-embedding-3-large")
print("current OpenAIEmbeddings().model=", myEmbeddings.model)

persist_dir = "../localdb/my_chroma_db_01"
myCollectionName = "my_collection"

vdb = Chroma.from_documents(
    splitedDocs,
    myEmbeddings
)
print("vdb적재문서수=", vdb._collection.count())


current OpenAIEmbeddings().model= text-embedding-ada-002
current OpenAIEmbeddings().model= text-embedding-3-large


BadRequestError: Error code: 400 - {'error': {'message': 'Requested 367501 tokens, max 300000 tokens per request', 'type': 'max_tokens_per_request', 'param': None, 'code': 'max_tokens_per_request'}}

In [7]:
#
# Step3-2: using package 'langchain_openai', not 'OpenAIEmbeddings'
#          from langchain.embeddings import OpenAIEmbeddings
#          'langchain.embeddings' deprecated, use 'langchain_openai'
#
#from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings


myEmbeddings = OpenAIEmbeddings()
print("current OpenAIEmbeddings().model=", myEmbeddings.model)
myEmbeddings = OpenAIEmbeddings(model="text-embedding-3-large")
print("current OpenAIEmbeddings().model=", myEmbeddings.model)

persist_dir = "../localdb/my_chroma_db_01"
myCollectionName = "my_collection"

#
# if u first create and add with splitedDocs to myEmbeddings, use Step3-2
# but if u want to use already Chroma data, use Step3-3
#


current OpenAIEmbeddings().model= text-embedding-ada-002
current OpenAIEmbeddings().model= text-embedding-3-large


In [8]:
#
# Step3-2: Loading splitedDocs to chroma
# chatgpt guide, use this code
#
from langchain_chroma import Chroma

def process_in_batches(documents, batch_size=100, reset_db=True):
    global vdb

    # -- 집어 넣을 문서 확인 --
    splitedDocsLen = len(splitedDocs)
    print(">> ready to insert splitedDocs, len = ", splitedDocsLen)

    if splitedDocsLen == 0:
        print("splitedDocs가 비어있습니다.")
        return 0

    # -- 문서 크기를 batch_size로 나누어 확인하기
    batchCount = splitedDocsLen // batch_size
    if splitedDocsLen % batch_size > 0:
        batchCount += 1
    print(">> batchsize is ", batch_size, "and batchCount:", batchCount)

    # -- 문서를 batch_size로 잘라서 집어 넣기
    for i, batch in enumerate(range(0, splitedDocsLen, batch_size)):
        print(">>", i, "th batch, ", batch, "..", batch + batch_size - 1, end="")
        batchDocs = splitedDocs[batch:batch + batch_size]
        vdb.add_documents(batchDocs)
        print(" ++ added batchDocs len:", len(batchDocs))
    print(">> vdb size:", len( vdb.get()["ids"] ))

def clear_before_addDoc():
    global vdb
    all_ids = vdb.get()["ids"]

    if ( len(all_ids) == 0 ):
        print(">> vdb size = 0 이므로 collection delete 하지 않음.")
        return

    # 컬렉션 내부 데이터 전체 삭제
    print(f">> vdb size = {len(all_ids)}, 벡터 컬렉션 데이터 삭제 시작...", end="")
    try:
        vdb._collection.delete(ids=all_ids)
        print("++벡터 컬렉션 데이터 삭제 완료.")
    except Exception as e:
        print("++벡터 컬렉션 삭제 실패:", e)
    print(">> vdb size = ", len( vdb.get()["ids"] ) )

# -- main body


# --- 기존(또는 새) 컬렉션 열기(임베딩 함수 전달) ---
vdb = Chroma(
    embedding_function=myEmbeddings,
    persist_directory=persist_dir,
    collection_name=myCollectionName
)

print("[START] vdb 크기=", len( vdb.get()["ids"] ) )
print(">>Persist Dir:", persist_dir)
print(">>myCollectionName:", myCollectionName)

clear_before_addDoc()
process_in_batches(splitedDocs, 100, True)

#print("vdb적재문서수=", vdb._collection.count())
print("[END] vdb 크기=", len( vdb.get()["ids"] ) )

[START] vdb 크기= 0
>>Persist Dir: ../localdb/my_chroma_db_01
>>myCollectionName: my_collection
>> vdb size = 0 이므로 collection delete 하지 않음.
>> ready to insert splitedDocs, len =  496
>> batchsize is  100 and batchCount: 5
>> 0 th batch,  0 .. 99 ++ added batchDocs len: 100
>> 1 th batch,  100 .. 199 ++ added batchDocs len: 100
>> 2 th batch,  200 .. 299 ++ added batchDocs len: 100
>> 3 th batch,  300 .. 399 ++ added batchDocs len: 100
>> 4 th batch,  400 .. 499 ++ added batchDocs len: 96
>> vdb size: 496
[END] vdb 크기= 496


In [10]:
#
# Step3-3: Use splitedDocs at chroma, already Chroma Data
#     only need: openAI API connection
#                myEmbeddings for Chroma
#
from langchain_chroma import Chroma

vdb = Chroma(
    embedding_function=myEmbeddings,
    persist_directory=persist_dir,
    collection_name=myCollectionName
)

print(">>Persist Dir:", persist_dir)
print(">>myCollectionName:", myCollectionName)
print(">>vdb 크기=", len( vdb.get()["ids"] ) )

>>Persist Dir: ../localdb/my_chroma_db_01
>>myCollectionName: my_collection
>>vdb 크기= 496


In [54]:
#
# Step4-1:
#
question = '북한의 교육과정'
resultDocs = vdb.similarity_search(question)
print('유사문서수:', len(resultDocs))
print( type(resultDocs), type(resultDocs[0]), type(resultDocs[0].page_content) )

유사문서수: 4
<class 'list'> <class 'langchain_core.documents.base.Document'> <class 'str'>


In [63]:
for resultDoc in resultDocs:
    print('---' * 10)
    print(resultDoc.metadata.get("page_label"), resultDoc.page_content[:100] )


------------------------------
284 309		북한의	학제는	2012년	전반적	의무교육(유치원	1년,	소학교	5년,	초급중학교	3년,	고급중학교	3년)으로	
개편되었는데,	학제개편	이전에는	초급중학교와	고급중학교를	통
------------------------------
42 2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에
------------------------------
352 2023 북한인권보고서
350
소학교 때는 김일성, 김정일, 김정숙의 어린 시절을 배우고, 초급중
학교에서는 혁명활동을 배우는데, 김정숙과 김정은의 내용을 학기
마다 번갈아 가며 
------------------------------
351 4. 교육권
349
IV. 경제적·사회적·문화적 권리 I. 발간개요V. 취약계층VI. 특별사안 II. 요약III. 시민적·정치적 권리
및 종교 집단 사이에 이해를 증진시킬 수 있도


In [66]:
#
# 4-2: other handle of ChromaDB
#
vdb2 = Chroma(
    persist_directory=persist_dir,
    embedding_function=myEmbeddings,
    collection_name=myCollectionName
)
print('문서의 수:', vdb2._collection.count())
print( type(vdb), type(vdb2) )

question = '북한의 교육과정'
resultDocs = vdb2.similarity_search(question)
print('유사문서수:', len(resultDocs))
print( type(resultDocs), type(resultDocs[0]), type(resultDocs[0].page_content) )

for resultDoc in resultDocs:
    print('---' * 10)
    print(resultDoc.metadata.get("page_label"), resultDoc.page_content[:100] )


문서의 수: 496
<class 'langchain_chroma.vectorstores.Chroma'> <class 'langchain_chroma.vectorstores.Chroma'>
유사문서수: 4
<class 'list'> <class 'langchain_core.documents.base.Document'> <class 'str'>
------------------------------
284 309		북한의	학제는	2012년	전반적	의무교육(유치원	1년,	소학교	5년,	초급중학교	3년,	고급중학교	3년)으로	
개편되었는데,	학제개편	이전에는	초급중학교와	고급중학교를	통
------------------------------
42 2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에
------------------------------
352 2023 북한인권보고서
350
소학교 때는 김일성, 김정일, 김정숙의 어린 시절을 배우고, 초급중
학교에서는 혁명활동을 배우는데, 김정숙과 김정은의 내용을 학기
마다 번갈아 가며 
------------------------------
351 4. 교육권
349
IV. 경제적·사회적·문화적 권리 I. 발간개요V. 취약계층VI. 특별사안 II. 요약III. 시민적·정치적 권리
및 종교 집단 사이에 이해를 증진시킬 수 있도


In [53]:
print( vdb is vdb2 )
print( vdb == vdb2 )
print( id(vdb) == id(vdb2), id(vdb), id(vdb2) )
print( vdb._collection.id == vdb2._collection.id, vdb._collection.id, vdb2._collection.id )
print('------')
print( type(vdb), type(db_from_file) )
print( type(vdb._client), type(db_from_file._client))

print('------')
print(vdb._collection.name, db_from_file._collection.name)
print(vdb._collection.name == db_from_file._collection.name) # True
print(vdb._client.list_collections() )
#print(vdb._client._settings.chroma_db_impl)
#print(db_from_file._client._settings.chroma_db_impl)# 둘 다 'duckdb+parquet'
#print(vdb._client._settings.persist_directory == db_from_file._client._settings.persist_directory)  # True


False
False
False 4455112272 4426322992
True ff88eb6d-51b2-41de-bc89-9d68063bce54 ff88eb6d-51b2-41de-bc89-9d68063bce54
------
<class 'langchain_chroma.vectorstores.Chroma'> <class 'langchain_chroma.vectorstores.Chroma'>
<class 'chromadb.api.client.Client'> <class 'chromadb.api.client.Client'>
------
my_collection my_collection
True
[Collection(name=my_collection)]


In [96]:
#
# 4-3
# find top 3, print similarity score
# List[Document] -> docs[0].metadata, docs[0].page_content
# List[Tuple[Document, float]] -> docs[0][0].metadata, docs[0][1]
question = '북한의 교육과정'
resultDocs = vdb2.similarity_search_with_relevance_scores(question, k=3)
print('유사문서수:', len(resultDocs))
print( type(resultDocs), type(resultDocs[0]) )
print( type(resultDocs[0][0]), type(resultDocs[0][0].page_content) )
print( type(resultDocs[0][1]), resultDocs[0][1] )

for resultDoc in resultDocs:
    print('---' * 10)
    print(resultDoc[1], resultDoc[0].page_content[:100])

for resultDoc, score in resultDocs:
    print('---' * 10)
    print(score, len(resultDoc.page_content) )

유사문서수: 3
<class 'list'> <class 'tuple'>
<class 'langchain_core.documents.base.Document'> <class 'str'>
<class 'float'> 0.5219001432461238
------------------------------
0.5219001432461238 309		북한의	학제는	2012년	전반적	의무교육(유치원	1년,	소학교	5년,	초급중학교	3년,	고급중학교	3년)으로	
개편되었는데,	학제개편	이전에는	초급중학교와	고급중학교를	통
------------------------------
0.36516962706617784 2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에
------------------------------
0.36082879669749535 2023 북한인권보고서
350
소학교 때는 김일성, 김정일, 김정숙의 어린 시절을 배우고, 초급중
학교에서는 혁명활동을 배우는데, 김정숙과 김정은의 내용을 학기
마다 번갈아 가며 
------------------------------
0.5219001432461238 187
------------------------------
0.36516962706617784 866
------------------------------
0.36082879669749535 867
